# Supplemental File: Imputation & Feature Selection

This notebook contains all trials and benchmarking work on finding the optimal imputation methods.

In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.metrics import f1_score
from itertools import product
from joblib import Parallel, delayed
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import KFold

In [2]:
df = pd.read_csv("data/training_v2.csv") # load the dataset
df_test = pd.read_csv("data/unlabeled.csv") # load the unlabeled test set

# specify response variable
target = "hospital_death"
y = df[target]
X = df.drop(columns=[target])
X_unlabeled = df_test

## 1. Feature Selection

In this section, we explored feature selection strategies to improve model stability, reduce noise, and streamline the downstream predictive workflow. Numerical, binary, and categorical features were examined separately to ensure that each variable was handled appropriately according to its data type. Initial screening included correlation checks, missingness assessment, and variance filtering to detect redundancy and identify features likely to contribute predictive value.

Although feature selection was later moved to occur post-imputation in the final pipeline, an initial round of preprocessing was still applied before imputation to remove features that were clearly uninformative or unsuitable for modeling. Specifically, we dropped identifier-like columns (`encounter_id`, `hospital_id`, `patient_id`, `icu_id`) as they carry no predictive signal, and removed numerical variables with more than 80% missingness or zero variance, as these would be difficult to impute and unlikely to add meaningful information. In total, 39 columns were removed during this initial filtering process.

In [3]:
# remove identifier columns
id_cols = ['encounter_id', 'hospital_id', 'patient_id', 'icu_id']
df.drop(columns=id_cols, inplace=True) # drop identifier columns

# identify numerical and categorical columns
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
bin_cols = [col for col in num_cols if df[col].nunique() == 2]
num_cols = num_cols.drop(bin_cols)
cat_cols = df.select_dtypes(include=["object", "category"]).columns

# identify columns with >80% missing values
missing_rate = df[num_cols].isnull().mean()
high_missing_cols = missing_rate[missing_rate > 0.8].index.tolist()

# identify columns with zero standard deviation
df_temp = df[num_cols].fillna(df[num_cols].mean())
zero_std_cols = df_temp.columns[df_temp.std() == 0].tolist()

# remove highly missing and zero std columns from numerical features
filtered_numeric_cols = [c for c in num_cols if c not in high_missing_cols + zero_std_cols]
print(f"Features filtered out (missing > 80% or SD=0): {len(high_missing_cols) + len(zero_std_cols)}")
num_cols = filtered_numeric_cols

Features filtered out (missing > 80% or SD=0): 35


## 2. Imputation

To evaluate imputation strategies for our dataset, we generated a ground-truth reference by imputing missing values and scaling continuous variables, then simulated missingness under a MCAR mechanism using randomized masking. We compared four imputation approaches applied only to continuous features - Mean, Median, KNN, and MICE - while binary and categorical variables were imputed consistently using most-frequent substitution for fairness. This is likely because many binary variables classified as MAR represent unlabeled or clinically insignificant results. Each method was wrapped inside a CompositeImputer class and assessed through 3-fold cross-validation, ensuring robustness against sampling variance.

Model performance was evaluated using RMSE measured only on artificially missing values to prevent information leakage. MICE achieved the lowest error across continuous variables (~0.61 RMSE) followed by KNN (~0.86 RMSE), whereas mean and median imputers performed similarly but worse overall (~0.98–1.00 RMSE). Binary columns showed identical RMSE across methods as expected, confirming that benchmarking primarily reflects numeric imputation quality. These results are consistent with literature: model-based imputers such as MICE generally outperform simpler univariate imputers when variables show correlation structure.

In [4]:
def make_random_mask(df_nona, missing_rate=0.2, seed=42):
    rng = np.random.default_rng(seed)
    mask = pd.DataFrame(
        rng.random(df_nona.shape) < missing_rate,
        index=df_nona.index,
        columns=df_nona.columns)
    return mask

def impute(df_masked, num_cols, cat_cols, num_method, cat_method):
    df_imputed = df_masked.copy()
    # impute numerical columns
    num_vals = num_method.fit_transform(df_imputed[num_cols])
    df_imputed[num_cols] = num_vals

    # inpute categorical columns
    cat_vals = cat_method.fit_transform(df_imputed[cat_cols])
    df_imputed[cat_cols] = np.array(cat_vals).reshape(-1, len(cat_cols))

    return df_imputed

# evaluate performance of 
def evaluate_imputation(df_nona, df_imputed, mask, num_cols, cat_cols):
    # evaluate numerical cols imputations using RMSE
    all_num = df_nona[num_cols]
    imp_num = df_imputed[num_cols]
    specific_num_mask = mask[num_cols]

    # extract only the values where the mask is True (the imputed positions)
    true_num_values = all_num.values[specific_num_mask.values]
    imp_num_values = imp_num.values[specific_num_mask.values]

    # calculate RMSE
    if len(true_num_values) > 0:
        rmse = np.sqrt(mean_squared_error(true_num_values, imp_num_values))
    else:
        rmse = 0 # handle case where no values were imputed

    # evaluate categorical cols imputations using accuracy
    cat_cols_to_evaluate = df_nona[cat_cols]
    imp_cat_cols = df_imputed[cat_cols]
    specific_cat_mask = mask[cat_cols]

    true_cat_values = cat_cols_to_evaluate.values[specific_cat_mask.values]
    imp_cat_values = imp_cat_cols.values[specific_cat_mask.values]

    if len(true_cat_values) > 0:
        acc = (true_cat_values == imp_cat_values).mean()
    else:
        acc = 1.0 # Or 0, depending on context
    
    f1_macro = f1_score(true_cat_values, imp_cat_values, average="macro")

    return rmse, acc, f1_macro

In [5]:
# 1. Create scaled ground truth dataset
# ---------------------------------------------------------
all_cols = list(num_cols) + list(bin_cols) + list(cat_cols)
X_full = df[all_cols].copy()

# 1a. fill NaNs to create "Truth" set
X_full[bin_cols] = SimpleImputer(strategy='most_frequent').fit_transform(X_full[bin_cols])
X_full[num_cols] = SimpleImputer(strategy='mean').fit_transform(X_full[num_cols])
X_full[cat_cols] = SimpleImputer(strategy='most_frequent').fit_transform(X_full[cat_cols])

# 1b. scale num_cols for KNN/MICE
scaler = StandardScaler()
X_full[num_cols] = scaler.fit_transform(X_full[num_cols])

# use a manageable sample size for the heavy CV benchmarking
X_ground_truth = X_full.sample(n=3000, random_state=42).reset_index(drop=True)

    


In [6]:
# 2. Specify imputers for categorical, binary, and continuous data
# ---------------------------------------------------------
# Imputer for Fixed Binary/Categorical Strategy (Mode)
class FixedModeImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.imputer_ = SimpleImputer(strategy='most_frequent').fit(X)
        return self
    def transform(self, X):
        return self.imputer_.transform(X)

# Orchestrator Class: Combines Fixed Binary/Cat Imputation with the Benchmarked Numeric Method
class CompositeImputer(BaseEstimator, TransformerMixin):
    def __init__(self, cont_imputer, numeric_cols, binary_cols, categorical_cols):
        self.num_cols = numeric_cols
        self.bin_cols = binary_cols
        self.cat_cols = categorical_cols
        
        # Benchmarked Imputer for Continuous (num_cols)
        self.cont_imputer = cont_imputer 
        
        # Fixed Imputer for Binary (bin_cols) and Categorical (cat_cols)
        self.bin_imputer = SimpleImputer(strategy='most_frequent', add_indicator=False)
        self.cat_imputer = SimpleImputer(strategy='most_frequent', add_indicator=False)
        self.feature_names_in_ = None
        
    def fit(self, X, y=None):
        self.feature_names_in_ = X.columns.tolist()
        
        # Fit Continuous (num_cols)
        self.cont_imputer.fit(X[self.num_cols])
        
        # Fit Fixed strategies
        self.bin_imputer.fit(X[self.bin_cols])
        self.cat_imputer.fit(X[self.cat_cols])
        return self
        
    def transform(self, X):
        # 1. Continuous (num_cols) -> Benchmarked Imputer (e.g., KNN/MICE)
        df_num = pd.DataFrame(self.cont_imputer.transform(X[self.num_cols]), 
                              columns=self.num_cols, index=X.index)
        
        # 2. Binary (bin_cols) -> Fixed Mode Imputation
        df_bin = pd.DataFrame(self.bin_imputer.transform(X[self.bin_cols]),
                              columns=self.bin_cols, index=X.index)
        
        # 3. Categorical (cat_cols) -> Fixed Mode Imputation
        df_cat = pd.DataFrame(self.cat_imputer.transform(X[self.cat_cols]),
                              columns=self.cat_cols, index=X.index)
        
        # Recombine and return in original order
        return pd.concat([df_num, df_bin, df_cat], axis=1)[self.feature_names_in_].values

In [ ]:
# 3. Define Imputers to Benchmark
# ---------------------------------------------------------
# We only benchmark the Continuous Imputer part, wrapping them in the Composite strategy
def create_composite(imputer_name):
    if imputer_name == "KNN":
        return CompositeImputer(KNNImputer(n_neighbors=5), num_cols, bin_cols, cat_cols)
    if imputer_name == "MICE":
        return CompositeImputer(IterativeImputer(max_iter=50, random_state=42), num_cols, bin_cols, cat_cols)
    if imputer_name == "Mean":
        return CompositeImputer(SimpleImputer(strategy='mean'), num_cols, bin_cols, cat_cols)
    if imputer_name == "Median":
        return CompositeImputer(SimpleImputer(strategy='median'), num_cols, bin_cols, cat_cols)
    return None

imputers_to_run = {
    "Composite_Mean": create_composite("Mean"),
    "Composite_Median": create_composite("Median"),
    "Composite_KNN": create_composite("KNN"),
    "Composite_MICE": create_composite("MICE"),
}

# 4. Cross-Validated Benchmarking Function
# ---------------------------------------------------------
def benchmark_cv(X_true, imputers_dict, n_splits=3, missing_rate=0.2):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    results = []
    
    print(f"Running {n_splits}-Fold CV Benchmark on {len(X_true)} rows...")
    
    for fold, (_, test_idx) in enumerate(kf.split(X_true), 1):
        X_fold = X_true.iloc[test_idx].copy()
        
        # Generate Random Mask (MCAR Simulation)
        rng = np.random.default_rng(fold)
        mask = pd.DataFrame(rng.random(X_fold.shape) < missing_rate, index=X_fold.index, columns=X_fold.columns)
        X_masked = X_fold.mask(mask)
        
        for name, imputer in imputers_dict.items():
            imputer.fit(X_masked)
            X_imp_df = pd.DataFrame(imputer.transform(X_masked), columns=X_masked.columns, index=X_masked.index)
            
            # Calculate RMSE only on the imputed numerical columns
            def calc_rmse(cols):
                if len(cols) == 0: return 0.0
                m = mask[cols].values
                if m.sum() == 0: return 0.0
                return np.sqrt(mean_squared_error(X_fold[cols].values[m], X_imp_df[cols].values[m]))
            
            results.append({
                "Method": name,
                "Fold": fold,
                "RMSE_Total_Num": calc_rmse(num_cols + bin_cols), # Total Numerical RMSE
                "RMSE_Num": calc_rmse(num_cols),                   # Continuous RMSE (Primary Benchmark)
                "RMSE_Bin": calc_rmse(bin_cols)                    # Binary RMSE (Fixed Strategy Check)
            })
        
    return pd.DataFrame(results)

# 5. Execute and Report
# ---------------------------------------------------------
df_results = benchmark_cv(X_ground_truth, imputers_to_run, n_splits=3)
summary = df_results.groupby("Method")[["RMSE_Total_Num", "RMSE_Num", "RMSE_Bin"]].agg(['mean', 'std'])

print("\nImputation Benchmark Results (Mean RMSE +/- Std Dev):")
print("Note: Continuous (num_cols) data is Z-score scaled.")
print(summary)

Running 3-Fold CV Benchmark on 3000 rows...


## 3. Summary
Overall, the imputation experiment demonstrates that MICE offers the most accurate recovery of continuous values, producing the lowest RMSE in every evaluation fold. Given this performance advantage and the presence of correlated features in the dataset,** MICE is selected as the default imputation strategy** moving forward. Mean/Median imputers remain useful as lightweight baselines and sensitivity comparisons, while KNN serves as a reasonable alternative balancing performance and computational cost. Future extensions may consider MAR/MNAR simulations or downstream model evaluation after imputation to assess whether reduced RMSE also translates into predictive improvement.